In [1]:
import sys
import os
import pandas as pd
import nltk
import seaborn as sns
import matplotlib
from pathlib import Path

# Usando o TkAGG para interagir com o gráfico
# Se quiser só plotar os gráficos normalmente comente a linha abaixo
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt
import spacy
import re
from nltk.stem import RSLPStemmer
import networkx as nx
from collections import defaultdict
from pyvis.network import Network
import numpy as np

nlp = spacy.load("pt_core_news_sm")
stemmer = RSLPStemmer()

# importando modulo customizado
print(os.getcwd())
# Coloque o seu path
sys.path.append(r'C:\Users\User\Documents\dumps\network-graph-whatsapp')  # Usando o diretório de trabalho atual

import Funcao

C:\Windows\system32


### Conectando ao banco e recuperando dados da tabela "hi_conversas"


In [2]:

import Banco
banco = Banco.CRUD()
resp = banco.fazer_requisicao('SELECT * FROM hi_conversas')
df = pd.DataFrame(resp)

### Mapeando as colunas recuperadas para nomes mais descritivos


In [3]:
mapa = {
    0: 'protocolo_index',
    1: 'protocolo',
    2: 'tipo_remetente',
    3: 'nome_remetente',
    4: 'demora',
    5: 'text',
    6: 'duracao',
    7: 'data',
    8: 'topico',
    9: 'score_score',
    10: 'text_eng',
    11: 'sentiment'
}

df = df.rename(columns=mapa)  # Renomeia as colunas de acordo com o mapa

### Limpeza e filtragem dos textos

In [4]:
# Mantendo apenas as colunas relevantes para análise
df = df[['protocolo', 'protocolo_index', 'text', 'tipo_remetente']]

df = df[df['tipo_remetente'] == 0].copy()


# Filtra as mensagens para remover aquelas cujo protocolo_index termina com "-00"
df = df[~df['protocolo_index'].str.endswith('-00')]

# Aplica a função de formatação e ordena o DataFrame pelo protocolo_index
df['protocolo_index'] = df['protocolo_index'].apply(Funcao.format_index)
df = df.sort_values(by='protocolo_index')
df = df.dropna(subset=['text'])

# Agrupa por protocolo e tipo_remetente, concatenando os textos em uma única string para cada grupo
df = df.groupby(['protocolo', 'tipo_remetente']).agg({
    'text': ' '.join
}).reset_index()

# Remove linhas com texto vazio ou NaN
df['text'] = df.text.apply(lambda x: Funcao.preprocess_text(x))
df['text'] = df.text.apply(lambda x: Funcao.remover_stopwords(x))
df = df[df['text'].str.strip().astype(bool)]  # Filtra textos não vazios

### Criação de dicionario para normalizar as palavras

In [5]:
def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join(token.lemma_ for token in doc)

def stemmer_text(text):
    return ' '.join(stemmer.stem(word) for word in text.split())

In [7]:

# Dividir o texto em listas de palavras
mapa = {
    0: 'palavra'
}

df['text_split'] = df['text'].apply(lambda x: x.split())


# Explodir a coluna para criar uma linha para cada palavra
df_exploded = df.explode('text_split').groupby("text_split").size().reset_index(name='repeteco').sort_values('repeteco',ascending=False)
df_exploded_grouped_filtered = df_exploded[df_exploded['repeteco'] >= 2]
df_exploded_grouped_filtered = df_exploded_grouped_filtered[~df_exploded_grouped_filtered['text_split'].str.contains(r'\d', regex=True)]
df_palavras = df_exploded_grouped_filtered
df_palavras = df_palavras.drop(columns='repeteco') 
df_palavras

,text_split
44686,pedido
38152,gostaria
31492,compra
48537,saber
49363,site
...,...
51586,vantajoso
31589,comrpa
27366,amarrotada
26816,agendas


In [8]:
# Função para lematizar tokens em uma string


df_compara = df_palavras
df_compara['text_base'] = df_compara["text_split"].apply(lemmatize_text)
df_compara['text_base'] = df_compara['text_base'].apply(lambda x: x.lower())
df_compara["text_base"] = df_compara['text_base'].apply(stemmer_text)

df_grouped = df_compara.groupby('text_base', as_index=False).agg({
    'text_split': 'first'  
})

mapa = {
    "text_split" : "final_correction"
}
df_grouped = df_grouped.rename(columns=mapa)
merged_df = pd.merge(df_compara, df_grouped[['text_base', 'final_correction']], on='text_base', how='left')
final_compare_df = merged_df
final_compare_df



,text_split,text_base,final_correction
0,pedido,ped,pedido
1,gostaria,gost,gostaria
2,compra,compr,compra
3,saber,sab,saber
4,site,sit,site
...,...,...,...
12982,vantajoso,vantaj,vantajoso
12983,comrpa,comrp,comrpa
12984,amarrotada,amarrot,amarrota
12985,agendas,agenr,agendas


In [9]:
dic_compare = final_compare_df.to_dict()
keys = dic_compare['text_base'].keys()
for k in keys:
    dic = {

        'text_split' : dic_compare['text_split'][k],
        'text_base' : dic_compare['text_base'][k],
        'final_correction':dic_compare['final_correction'][k]
    }
    query = Funcao.montar_query_insert(dic,'dicionario_simplificante','n')
    banco.fazer_requisicao(query)


INSERT INTO dicionario_simplificante (text_split, text_base, final_correction) VALUES ('pedido', 'ped', 'pedido');
MySQL Error: (1062, "Duplicate entry 'pedido' for key 'dicionario_simplificante.PRIMARY'")
INSERT INTO dicionario_simplificante (text_split, text_base, final_correction) VALUES ('gostaria', 'gost', 'gostaria');
MySQL Error: (1062, "Duplicate entry 'gostaria' for key 'dicionario_simplificante.PRIMARY'")
INSERT INTO dicionario_simplificante (text_split, text_base, final_correction) VALUES ('compra', 'compr', 'compra');
MySQL Error: (1062, "Duplicate entry 'compra' for key 'dicionario_simplificante.PRIMARY'")
INSERT INTO dicionario_simplificante (text_split, text_base, final_correction) VALUES ('saber', 'sab', 'saber');
MySQL Error: (1062, "Duplicate entry 'saber' for key 'dicionario_simplificante.PRIMARY'")
INSERT INTO dicionario_simplificante (text_split, text_base, final_correction) VALUES ('site', 'sit', 'site');
MySQL Error: (1062, "Duplicate entry 'site' for key 'dicion

### Coletando dicionario de palavras do banco MySQL para ajustar os textos do DataFrame



In [10]:
resp  = banco.fazer_requisicao("select * from dicionario_simplificante")
df_dicionario = pd.DataFrame(resp)
mapa = {

    0: "text_split",
    1: "text_base",
    2: "final_correction"
}
df_dicionario = df_dicionario.rename(columns=mapa)


### Criando Função para ajustar os textos do DataFrame


In [11]:

def replace_words_in_text(text, replacements):
    # Divida o texto em palavras
    words = text.split()
    # Substitua as palavras com base no DataFrame de substituições
    corrected_words = [replacements.get(word, word) for word in words]
    # Junte as palavras de volta em um texto
    return ' '.join(corrected_words)

replacement_dict = dict(zip(df_dicionario['text_split'], df_dicionario['final_correction']))


In [12]:
def replace_words(text):
    # Dicionário de substituições dentro da função
    alter_words = {

        ' gostaria ': ' quero ',
        ' nao ': ' nao ',
        ' preciso ': ' quero ',
        ' queria ': ' quero ',
        ' desejo ': ' quero ',
        ' negativo ':' nao ',
        ' camiseta': ' camisa ',
        ' quero ':' quero ',
        ' malha fria ':' malha-fria ',
        ' azul marinho ': ' azul-marinho ',
        ' dry fit ': ' dry-fit ',
        ' s ': ' ',
        ' pretendo ': ' quero ',
        ' almejo ': ' quero ',
        ' necessito ': ' quero ',
        ' n ':' nao ',
        ' cartao credito ': ' cartao-credito ',
        ' manga longa ': ' manga-longa ',
        ' loja mirante ': ' loja-mirante ',
        ' nota fiscal ' : ' nf '
        
    }
    
    # Substituindo palavras de acordo com o dicionário
    for word, replacement in alter_words.items():
        text = text.replace(word, replacement)
    return text

### Aplica as funções 


In [13]:
df['text'] = df['text'].apply(lambda x: replace_words_in_text(x, replacement_dict))
df['text'] = df['text'].apply(lambda x: replace_words(x))


### Remove palavras que contenham ou sejam compostas por numeros

In [14]:
def remove_words_with_numbers(text):
    # Regex para remover palavras que contenham números
    return re.sub(r'\b\w*\d\w*\b', '', text).strip()

# Aplicando a função em toda a coluna de textos
df['text'] = df['text'].apply(lambda x: remove_words_with_numbers(x))

### Cria a relação entre as palavras de cada texto

In [15]:
def build_cooccurrence_graph(texts):
    G = nx.Graph()  # Criar um grafo vazio
    for text in texts:
        words = text.split()  # Tokenizar o texto
        for i in range(len(words) - 1):
            word_pair = (words[i], words[i + 1])  # Par de palavras consecutivas
            if G.has_edge(*word_pair):
                G[word_pair[0]][word_pair[1]]['weight'] += 1  # Fortalecer a conexão existente
            else:
                G.add_edge(*word_pair, weight=1)  # Adicionar nova conexão
    return G

G = build_cooccurrence_graph(df['text'])


### Ajusta os dados para Plotar

In [16]:


# Obter todas as arestas e seus pesos
edges = G.edges(data=True)

# Filtrar para remover arestas entre palavras iguais
filtered_edges = [(u, v, data) for u, v, data in edges if u != v]

# Ordenar as arestas por peso em ordem decrescente
sorted_edges = sorted(filtered_edges, key=lambda x: x[2]['weight'], reverse=True)

# Manter apenas as 100 conexões mais fortes (ou menos se houver menos de 100)
top_edges = sorted_edges[:100]

# Criar um novo grafo com apenas as arestas mais fortes
G_filtered = nx.Graph()
G_filtered.add_edges_from((u, v, data) for u, v, data in top_edges)

# Aplicar normalização e escalonamento logarítmico aos pesos das arestas
edge_weights = np.array([data['weight'] for u, v, data in G_filtered.edges(data=True)])
scaled_edge_weights = np.log1p(edge_weights)

# Definir uma largura máxima para as arestas e limitar as larguras escaladas das arestas
max_edge_width = 3
scaled_edge_weights = np.clip(scaled_edge_weights, 0, max_edge_width)

# Criar um mapeamento das arestas para suas larguras limitadas
edge_widths = { (u, v): w for (u, v), w in zip(G_filtered.edges(), scaled_edge_weights) }

# Layout para visualização
pos = nx.spring_layout(G_filtered, k=1.0)  # Ajuste o valor de k conforme necessário

# Aumentar o tamanho dos nós
node_size = 700  # Ajuste conforme necessário

### Desenha o Network Graph

In [17]:
# Desenhar o grafo
plt.figure(figsize=(14, 12))  # Aumentar o tamanho da figura
nx.draw(G_filtered, pos, with_labels=False, node_color='skyblue', node_size=node_size, font_size=10,
        width=[edge_widths[(u, v)] for u, v in G_filtered.edges()])

# Desenhar rótulos dos nós
labels = {node: node for node in G_filtered.nodes()}  
nx.draw_networkx_labels(G_filtered, pos, labels=labels, font_size=12, font_family='sans-serif', 
                        bbox=dict(facecolor='white', alpha=0.5))

# Desenhar rótulos das arestas
weights = nx.get_edge_attributes(G_filtered, 'weight')
nx.draw_networkx_edge_labels(G_filtered, pos, edge_labels={(u, v): f'{d["weight"]}' for u, v, d in top_edges})

plt.title('Filtered Graph with Top 100 Strongest Connections (No Self-Loops)')
plt.show()